Notebook purpose: evaluate how efficiently we could search for catalysts using the ML model under various constraints.

The most conspicuous constraint is to find a set number of active catalysts without any unnecessary DFT calculations
What is unnecessary? --> 100% of O2 binding calculations are to actual binding sites
So we can accept a model with lower accuracy as long as it has no false positives --> only a small penalty for false negatives

Let's say we're only willing to run 5 DFT O2 binding calculations, and we want basically all of them to show that we found active sites. We'd probably want each of these to be per catalyst, to show that we've found 5 unique active catalysts. Assuming we're working with 10% of the data as a "test" set, that's about 27 calalysts, so we want to pick the ones that the model is most confident have at least 1 site that binds O2.

Really, this is a question of whether the active sites for a set of catalysts are most likely to actually be binding
Can order by log-loss and take that as an estimate of uncertainty (is that a fair expectation?)


In [13]:
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.model_selection import GroupShuffleSplit

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix

In [2]:
from ngcc_ml import data_tools
from ngcc_ml import skl_tools

RDKit WARNING: [09:14:14] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
df = pd.read_csv("/home/nricke/work/ngcc_ml/DidItBindv5.csv")

In [66]:
df.columns

Index(['Unnamed: 0', 'Atom Number', 'Catalyst Name', 'CatalystO2File',
       'Element', 'SpinDensity', 'ChElPGPositiveCharge', 'ChElPGNeutralCharge',
       'ChargeDifference', 'Doesitbind', 'BondLength', 'IonizedFreeEnergy',
       'IonizationEnergy', 'BindingEnergy', 'NeutralFreeEnergy', 'OrthoOrPara',
       'Meta', 'FartherThanPara', 'DistanceToN', 'AverageBondLength',
       'BondLengthRange', 'NumberOfHydrogens', 'AromaticSize', 'IsInRingSize6',
       'IsInRingSize5', 'NeighborSpinDensity', 'NeighborChElPGCharge',
       'NeighborChargeDifference', 'AromaticExtent', 'RingEdge',
       'NumNitrogens', 'NumHeteroatoms', 'ring_nitrogens',
       'atom_plane_deviation', 'ring_plane_deviation', 'charge'],
      dtype='object')

In [36]:
feature_cols = {"SpinDensity", "ChElPGNeutralCharge", "ChargeDifference", "IonizationEnergy", "OrthoOrPara", "Meta", "FartherThanPara", "DistanceToN", "AverageBondLength",  "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "NeighborSpinDensity", 'NeighborChElPGCharge', 'NeighborChargeDifference', "AromaticExtent", "RingEdge", "NumNitrogens", "NumHeteroatoms", "charge", "atom_plane_deviation", "ring_plane_deviation", "ring_nitrogens"}
not_scaled_cols = {"OrthoOrPara", "Meta", "FartherThanPara", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "RingEdge", "NumNitrogens", "NumHeteroatoms", "ring_nitrogens", "charge"}
df_scale = data_tools.process_data(df, scaledCols=list(feature_cols - not_scaled_cols))
train_inds, test_inds = next(GroupShuffleSplit(test_size=0.10, n_splits=2, random_state = 6).split(df, groups=df['Catalyst Name']))
train = df.iloc[train_inds]
test = df.iloc[test_inds]
X_train_group = train[feature_cols]
y_train_group = train["Doesitbind"].astype("int")
X_test_group = test[feature_cols]
y_test_group = test["Doesitbind"].astype("int")

/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [37]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=100, class_weight={0:1, 1:10})
rfc.fit(X_train_group, y_train_group)
print('Accuracy of RFC on test set: {:.3f}'.format(rfc.score(X_test_group, y_test_group)))
print('Accuracy of RFC on training set: {:.3f}'.format(rfc.score(X_train_group, y_train_group)))
y_pred_group = rfc.predict(X_test_group)
print(confusion_matrix(y_test_group, y_pred_group))

Accuracy of RFC on test set: 0.956
Accuracy of RFC on training set: 1.000
[[354   6]
 [ 13  60]]


In [39]:
len(test["Catalyst Name"].unique())

27

In [42]:
p = rfc.predict_proba(X_test_group)

In [48]:
p

array([[1.        , 0.        ],
       [0.285     , 0.715     ],
       [0.98      , 0.02      ],
       [0.372     , 0.628     ],
       [0.983     , 0.017     ],
       [0.227     , 0.773     ],
       [0.985     , 0.015     ],
       [1.        , 0.        ],
       [0.063     , 0.937     ],
       [0.944     , 0.056     ],
       [0.122     , 0.878     ],
       [0.989     , 0.011     ],
       [0.203     , 0.797     ],
       [1.        , 0.        ],
       [0.218     , 0.782     ],
       [0.79704762, 0.20295238],
       [0.319109  , 0.680891  ],
       [0.969     , 0.031     ],
       [0.00500143, 0.99499857],
       [0.997     , 0.003     ],
       [0.975     , 0.025     ],
       [0.997     , 0.003     ],
       [0.93947368, 0.06052632],
       [0.998     , 0.002     ],
       [0.87747368, 0.12252632],
       [0.978     , 0.022     ],
       [0.74404762, 0.25595238],
       [0.929     , 0.071     ],
       [0.33201639, 0.66798361],
       [0.951     , 0.049     ],
       [0.

In [49]:
y_pred_group

array([0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

In [50]:
test["predict_proba"] = p[:,1]
test["prediction"] = y_pred_group

/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [52]:
test["Doesitbind"] = test["Doesitbind"].astype('int')

/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
test[test["Doesitbind"] != test["prediction"]]

,Unnamed: 0,Atom Number,Catalyst Name,CatalystO2File,Element,SpinDensity,ChElPGPositiveCharge,ChElPGNeutralCharge,ChargeDifference,Doesitbind,...,AromaticExtent,RingEdge,NumNitrogens,NumHeteroatoms,ring_nitrogens,atom_plane_deviation,ring_plane_deviation,charge,predict_proba,prediction
113,113,3,sf10x0,sf10x0O2-2_optsp_a0m2.out,C,0.278266,0.283251,0.126444,-0.156807,0,...,16,1,1,1,1,1.465000e-07,1.199000e-07,0,0.680891,1
121,121,11,sf10x0,sf10x0O2-10_optsp_a0m2.out,C,0.018886,-0.155541,-0.205475,-0.049934,1,...,16,2,1,1,1,1.141000e-07,1.199000e-07,0,0.122526,0
124,124,14,sf10x0,sf10x0O2-13_optsp_a0m2.out,C,0.092944,-0.300849,-0.336655,-0.035806,1,...,16,2,1,1,1,0.000000e+00,1.199000e-07,0,0.071000,0
126,126,16,sf10x0,sf10x0O2-15_optsp_a0m2.out,C,-0.109755,-0.304871,-0.322938,-0.018067,1,...,16,2,1,1,1,8.320000e-08,1.199000e-07,0,0.049000,0
229,229,9,sf11x0,sf11x0O2-8_optsp_a0m2.out,C,0.320678,0.294529,0.131280,-0.163249,1,...,20,1,1,1,1,2.418900e-06,2.658300e-06,0,0.297000,0
793,793,14,sf14x0,sf14x0O2-13_optsp_a0m2.out,C,0.268839,0.393293,0.280550,-0.112743,0,...,18,2,2,2,2,2.800000e-08,3.269000e-07,0,0.580000,1
1602,1602,1,sf196x0,sf196x0O2-0_optsp_a0m2.out,C,-0.059582,-0.274116,-0.311125,-0.037009,1,...,20,2,1,1,1,1.056000e-07,1.818000e-07,0,0.216000,0
1608,1608,7,sf196x0,sf196x0O2-6_optsp_a0m2.out,C,0.181557,-0.045219,-0.129107,-0.083888,1,...,20,2,1,1,1,2.413000e-07,1.818000e-07,0,0.150000,0
3082,3082,2,sf282x0,sf282x0O2-1_optsp_a0m2.out,C,0.156007,-0.294074,-0.337982,-0.043908,1,...,12,2,1,2,1,4.122000e-07,1.875000e-07,0,0.169000,0
3084,3084,4,sf282x0,sf282x0O2-3_optsp_a0m2.out,C,0.329151,0.143856,0.001354,-0.142502,0,...,12,1,1,2,1,6.920000e-08,1.875000e-07,0,0.628000,1


In [58]:
# were the most confident active site predictions actually active?
test_sort = test.sort_values(by="predict_proba", ascending=False)[["Catalyst Name", "Doesitbind", "prediction", "predict_proba"]]

In [61]:
list(test_sort["Doesitbind"]).index(0)

37

In [63]:
len(test_sort.head(36)["Catalyst Name"].unique())

21

In [64]:
test.columns

Index(['Unnamed: 0', 'Atom Number', 'Catalyst Name', 'CatalystO2File',
       'Element', 'SpinDensity', 'ChElPGPositiveCharge', 'ChElPGNeutralCharge',
       'ChargeDifference', 'Doesitbind', 'BondLength', 'IonizedFreeEnergy',
       'IonizationEnergy', 'BindingEnergy', 'NeutralFreeEnergy', 'OrthoOrPara',
       'Meta', 'FartherThanPara', 'DistanceToN', 'AverageBondLength',
       'BondLengthRange', 'NumberOfHydrogens', 'AromaticSize', 'IsInRingSize6',
       'IsInRingSize5', 'NeighborSpinDensity', 'NeighborChElPGCharge',
       'NeighborChargeDifference', 'AromaticExtent', 'RingEdge',
       'NumNitrogens', 'NumHeteroatoms', 'ring_nitrogens',
       'atom_plane_deviation', 'ring_plane_deviation', 'charge',
       'predict_proba', 'prediction'],
      dtype='object')